In [ ]:
from abc import ABC, abstractmethod
import json

class DataSource(ABC):
    @abstractmethod
    def load_data(self):
        pass

    @abstractmethod
    def process_data(self):
        pass

    @abstractmethod
    def analyze(self):
        pass


class JsonFileDataSource(DataSource):
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = []

    def load_data(self):
        with open(self.filepath, 'r') as file:
            self.data = json.load(file, object_hook=self.validate_and_filter)

    def process_data(self):
        return self.data

    def analyze(self):
        self.load_data()
        return self.process_data()

    @staticmethod
    def validate_and_filter(record):
        required_keys = {"product", "quantity", "price"}
        if not isinstance(record, dict):
            raise ValueError(f"Ошибка: запись должна быть объектом. Пропущено: {record}")
        if not required_keys.issubset(record.keys()):
            raise ValueError(f"Ошибка: запись должна содержать ключи {required_keys}. Пропущено: {record}")
        if not isinstance(record["product"], str):
            raise ValueError(f"Ошибка: 'product' должен быть строкой. Пропущено: {record}")
        if not isinstance(record["quantity"], int):
            raise ValueError(f"Ошибка: 'quantity' должен быть целым числом. Пропущено: {record}")
        if not isinstance(record["price"], int | float):
            raise ValueError(f"Ошибка: 'price' должен быть числом. Пропущено: {record}")
        return record

class TextFileDataSource(DataSource):
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = []

    def load_data(self):
        with open(self.filepath, 'r') as file:
            file.readline()
            for line in file:
                line = line.strip()
                if line:
                    self.data.append(line)

    def process_data(self):
        if not self.data:
            raise ValueError('No data')
        result = []
        for line in self.data:
            product, quantity, price = line.split('|')
            result.append({
                "product": product,
                "quantity": int(quantity),
                "price": float(price)
            })
        self.data = result
        return self.data

    def analyze(self):
        self.load_data()
        return self.process_data()


def merge_data(*data_processors):
    result = []
    for data in data_processors:
        result.extend(data.analyze())

    return result


text_processor = TextFileDataSource('sales.txt')
json_processor = JsonFileDataSource('sales.json')

result = merge_data(text_processor, json_processor)
print(result)